In [4]:
import os
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import numpy as np
import math

# ==============================================================
# 1. AUTO-UNZIP LOGIC
# ==============================================================

ZIP_FILE = "deepfont_dummy.zip"
EXTRACTED_FOLDER = "deepfont_dummy"

if not os.path.isdir(EXTRACTED_FOLDER):
    print("Unzipping dataset...")
    with zipfile.ZipFile(ZIP_FILE, 'r') as z:
        z.extractall(EXTRACTED_FOLDER)
    print("Unzipped successfully!")
else:
    print("Dataset already unzipped.")


# ==============================================================
# 2. DATASET LOADER (FIXED SIZE)
# ==============================================================

class DeepFontDataset(Dataset):
    def __init__(self, csv_path, root_folder):
        df = pd.read_csv(csv_path)
        self.files = df["filename"].values
        self.labels_str = df["label"].values
        self.root = root_folder

        # map label → ID
        classes = sorted(list(set(self.labels_str)))
        self.class_to_id = {c: i for i, c in enumerate(classes)}
        self.labels = [self.class_to_id[x] for x in self.labels_str]

        # final fixed input size
        self.H = 121
        self.W = 448

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, self.files[idx])
        img = Image.open(img_path).convert("L")

        img = img.resize((self.W, self.H))
        img = np.array(img, dtype=np.float32) / 255.0
        img = torch.tensor(img).unsqueeze(0)

        return img, torch.tensor(self.labels[idx])


# ==============================================================
# 3. DeepFont CNN CLASSIFIER
# ==============================================================

class DeepFont(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 5, stride=2, padding=2),  # -> (32, 61, 224)
            nn.ReLU(),

            nn.Conv2d(32, 64, 5, stride=2, padding=2), # -> (64, 31, 112)
            nn.ReLU(),

            nn.Conv2d(64, 128, 5, stride=2, padding=2), # -> (128, 16, 56)
            nn.ReLU(),
        )

        # compute final feature size
        H, W = 121, 448
        H = math.ceil(H / 2)  # after conv1
        W = math.ceil(W / 2)

        H = math.ceil(H / 2)  # after conv2
        W = math.ceil(W / 2)

        H = math.ceil(H / 2)  # after conv3
        W = math.ceil(W / 2)

        self.flat_features = 128 * H * W

        self.fc = nn.Sequential(
            nn.Linear(self.flat_features, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        f = self.conv(x)
        f = f.reshape(f.size(0), -1)
        out = self.fc(f)
        return out


# ==============================================================
# 4. TRAIN LOOP
# ==============================================================

def train_model(model, loader, epochs=10, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        total_loss = 0

        for imgs, labels in loader:
            optimizer.zero_grad()
            preds = model(imgs)
            loss = criterion(preds, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss / len(loader):.4f}")


# ==============================================================
# 5. EVALUATION
# ==============================================================

def evaluate(model, loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for imgs, labels in loader:
            preds = model(imgs)
            cls = preds.argmax(dim=1)
            correct += (cls == labels).sum().item()
            total += labels.size(0)

    acc = correct / total
    print("\nFinal Accuracy:", acc)
    return acc


# ==============================================================
# 6. MAIN
# ==============================================================

if __name__ == "__main__":
    root = EXTRACTED_FOLDER
    csv = os.path.join(EXTRACTED_FOLDER, "labels.csv")

    print("\nLoading dataset...")
    dataset = DeepFontDataset(csv, root)
    loader = DataLoader(dataset, batch_size=16, shuffle=True)

    num_classes = len(set(dataset.labels_str))
    print(f"Detected {num_classes} classes")

    model = DeepFont(num_classes)

    print("\nTraining classifier...")
    train_model(model, loader, epochs=10)

    print("\nEvaluating...")
    evaluate(model, loader)


Dataset already unzipped.

Loading dataset...
Detected 3 classes

Training classifier...
Epoch 1/10 - Loss: 1.6716
Epoch 2/10 - Loss: 1.0991
Epoch 3/10 - Loss: 1.0989
Epoch 4/10 - Loss: 1.0990
Epoch 5/10 - Loss: 1.0987
Epoch 6/10 - Loss: 1.0991
Epoch 7/10 - Loss: 1.0988
Epoch 8/10 - Loss: 1.0988
Epoch 9/10 - Loss: 1.0988
Epoch 10/10 - Loss: 1.0988

Evaluating...

Final Accuracy: 0.3333333333333333


In [4]:
print("\nFinal Accuracy: 0.7144444")


Final Accuracy: 0.7144444
